<a href="https://colab.research.google.com/github/sethsot/P5_Sentiment-Analysis/blob/main/P5_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

INSTALLATIONS

In [1]:
!pip install -q datasets transformers huggingface_hub accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 70.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 66.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 26.9 MB/s eta 0:00:00


In [2]:
!pip install transformers
!pip install datasets

In [3]:
import os
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from google.colab import drive
import torch

from sklearn.model_selection import train_test_split
from datasets import load_dataset, load_metric

from transformers import AutoTokenizer, TrainingArguments, Trainer, AutoModelForSequenceClassification

from huggingface_hub import notebook_login

import warnings

warnings.filterwarnings('ignore')


In [4]:
torch.cuda.is_available()

True

In [5]:
#set random seeds for Pytorch
seed=12
torch.manual_seed(seed)

# further set seeds for GPU operations if using CUDA
if torch.cuda.is_available():
  torch.backends.cudnn.deterministic=True
  torch.backends.cudnn.benchmark=False
  torch.cuda.manual_seed_all(seed)

LOAD DATASET


In [6]:
#Mount the googe drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [7]:
#Navigate to the Colab data folder
%cd drive/MyDrive/Dataset

/content/drive/MyDrive/Dataset


In [8]:
# Disabe W&B
os.environ["WANDB_DISABLED"] = "true"

In [9]:
#load train and test dataset
df=pd.read_csv("/content/drive/MyDrive/Dataset/Train.csv")
test=pd.read_csv("/content/drive/MyDrive/Dataset/Test.csv")

In [10]:
df.head()

,tweet_id,safe_text,label,agreement
0,CL1KWCMY,Me &amp; The Big Homie meanboy3000 #MEANBOY #M...,0.0,1.0
1,E3303EME,I'm 100% thinking of devoting my career to pro...,1.0,1.0
2,M4IVFSMS,"#whatcausesautism VACCINES, DO NOT VACCINATE Y...",-1.0,1.0
3,1DR6ROZ4,I mean if they immunize my kid with something ...,-1.0,1.0
4,J77ENIIE,Thanks to <user> Catch me performing at La Nui...,0.0,1.0


In [11]:
test.head()

,tweet_id,safe_text
0,00BHHHP1,<user> <user> ... &amp; 4 a vaccine given 2 he...
1,00UNMD0E,Students starting school without whooping coug...
2,01AXPTJF,"I'm kinda over every ep of <user> being ""rippe..."
3,01HOEQJW,How many innocent children die for lack of vac...
4,01JUKMAO,"CDC eyeing bird flu vaccine for humans, though..."


In [12]:
# Eliminate rows containing NaN values
df = df[~df.isna().any(axis=1)]
df.isna().sum()

tweet_id     0
safe_text    0
label        0
agreement    0
dtype: int64

In [13]:
# Eliminate rows containing NaN values
test = test[~test.isna().any(axis=1)]
test.isna().sum()

tweet_id     0
safe_text    0
dtype: int64

In [14]:
df.columns


Index(['tweet_id', 'safe_text', 'label', 'agreement'], dtype='object')

In [15]:
df['label'].unique()

array([ 0.,  1., -1.])

In [16]:
# Split the train data => {train, eval}
train, eval = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

In [17]:
print(f"new dataframe shapes: train is {train.shape}, eval is {eval.shape}")

new dataframe shapes: train is (7999, 4), eval is (2000, 4)


In [18]:
# Save splitted subsets
train.to_csv("/content/drive/MyDrive/Dataset/train_subset.csv", index=False)
eval.to_csv("/content/drive/MyDrive/Dataset/eval_subset.csv", index=False)


In [19]:
dataset = load_dataset('csv',
                        data_files={'train': '/content/drive/MyDrive/Dataset/train_subset.csv',
                        'eval': '/content/drive/MyDrive/Dataset/eval_subset.csv'}, encoding = "ISO-8859-1")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating eval split: 0 examples [00:00, ? examples/s]

In [20]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

In [21]:
def transform_labels(label):

    label = label['label']
    num = 0
    if label == -1.0: #'Negative'
        num = 0
    elif label == 0.0: #'Neutral'
        num = 1
    elif label == 1.0: #'Positive'
        num = 2

    return {'labels': num}

def tokenize_data(example):
  return tokenizer(example['safe_text'], padding='max_length')

# Change the tweets to tokens that the models can exploit
dataset = dataset.map(tokenize_data, batched=True)

# Transform	labels and remove the useless columns
remove_columns = ['tweet_id', 'label', 'safe_text', 'agreement']
dataset = dataset.map(transform_labels, remove_columns=remove_columns)

Map:   0%|          | 0/7999 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7999 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [22]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 7999
    })
    eval: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 2000
    })
})

In [23]:
dataset['train']

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 7999
})

In [24]:
# Set up the training arguments
training_args = TrainingArguments(
    output_dir='./output',
    save_total_limit=2,
    evaluation_strategy="steps",  # Choose "steps" or "epoch" based on your preference.
    eval_steps=500,  # The number of training steps between evaluations.
    load_best_model_at_end=True,
    num_train_epochs=3,  # Set the number of training epochs to 3.
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [25]:
# Loading a pretrain model while specifying the number of labels in our dataset for fine-tuning
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=3)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
train_dataset = dataset['train'].shuffle(seed=10)
eval_dataset = dataset['eval'].shuffle(seed=10)

In [27]:
trainer = Trainer(
    model=model, args=training_args, train_dataset=train_dataset, eval_dataset=eval_dataset
)

In [28]:
# Launch the learning process: training
trainer.train()

Step,Training Loss,Validation Loss
500,0.761700,0.774842
1000,0.690200,0.644997
1500,0.566700,0.766304
2000,0.544000,0.593688


Step,Training Loss,Validation Loss
500,0.761700,0.774842
1000,0.690200,0.644997
1500,0.566700,0.766304
2000,0.544000,0.593688
2500,0.376600,0.806636
3000,0.334700,0.876082


TrainOutput(global_step=3000, training_loss=0.5456684773763021, metrics={'train_runtime': 2695.4575, 'train_samples_per_second': 8.903, 'train_steps_per_second': 1.113, 'total_flos': 6313932685218816.0, 'train_loss': 0.5456684773763021, 'epoch': 3.0})

In [29]:
metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [30]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

In [31]:
# Launch the final evaluation
trainer.evaluate()

{'eval_loss': 0.5936880111694336,
 'eval_accuracy': 0.7555,
 'eval_runtime': 66.8741,
 'eval_samples_per_second': 29.907,
 'eval_steps_per_second': 3.738}